# LangChain 0.3.27 Tutorial: Real-World Applications with OpenRouter

## Table of Contents
1. [Setup & Installation](#setup--installation)
2. [Basic Configuration](#basic-configuration)
3. [Scenario 1: Customer Support Chatbot](#scenario-1-customer-support-chatbot)
4. [Scenario 2: Document Analysis & Summarization](#scenario-2-document-analysis--summarization)
5. [Scenario 3: Code Review Assistant](#scenario-3-code-review-assistant)
6. [Scenario 4: Content Generation Pipeline](#scenario-4-content-generation-pipeline)
7. [Scenario 5: Data Analysis & Insights](#scenario-5-data-analysis--insights)
8. [Advanced Features](#advanced-features)

## Setup & Installation

### Step 1: Install Required Packages

First, let's install LangChain and required dependencies in your Google Colab environment.

In [ ]:
# Install LangChain 0.3.27 and dependencies
!pip install langchain==0.3.27
!pip install langchain-openai
!pip install python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 1.8 MB/s eta 0:00:00


In [ ]:
# Install additional packages for document processing
!pip install pypdf2
!pip install chromadb
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 29.3 MB/s eta 

In [ ]:
# Install packages for web scraping and utilities
!pip install beautifulsoup4
!pip install requests
!pip install pandas numpy

In [ ]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00


### Step 2: Import Required Libraries

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate

In [ ]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

## Basic Configuration

### Step 3: Configure OpenRouter

First, you need to get an API key from [OpenRouter](https://openrouter.ai/). Then set it up in Colab:


In [ ]:
# Set your OpenRouter API key (replace with your actual key)
import getpass
api_key = getpass.getpass("Enter your OpenRouter API key: ")
#os.environ["OPENAI_API_KEY"] = "sk-or-v1-d364066ceb3339067def1c2ca462d2f8c5e760aac5b68632b124a7d198ab8bebi"
os.environ["OPENAI_API_KEY"] = api_key

Enter your OpenRouter API key: ··········


In [ ]:
# Configure the LLM with OpenRouter
llm = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    model="openai/gpt-oss-20b:free",
    temperature=0.7
)

### Step 4: Test Basic Connection

In [ ]:
# Test the connection
response = llm.invoke([HumanMessage(content="Hello! Can you confirm the connection is working?")])
print(response.content)

Hello! Yes, the connection is working—happy to help with whatever you need.


## Scenario 1: Customer Support Chatbot

### Real-World Application: E-commerce Customer Service

Let's build a customer support chatbot for an online electronics store.

In [ ]:
# Create a system prompt for customer support
system_prompt = """You are a helpful customer support agent for TechStore,
an online electronics retailer. You should be friendly, professional,
and knowledgeable about common issues like orders, returns, and products."""

In [ ]:
# Create a conversation memory
memory = ConversationBufferMemory(return_messages=True)

/tmp/ipython-input-1349388406.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)


In [ ]:
# Create the customer support chain
support_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

In [ ]:
from langchain_core.runnables import RunnableSequence
from operator import itemgetter

# Create the customer support chain with memory integration
# This pattern loads memory, formats the prompt, invokes the LLM, and saves the context
support_chain = (
    {
        "input": itemgetter("input"),
        "chat_history": lambda x: memory.load_memory_variables({})["history"]
    }
    | support_prompt
    | llm
)

# Note: Saving the context back to memory after the response
# is handled separately when invoking the chain in the next step.

In [ ]:
# Test the customer support bot
response = support_chain.invoke({"input": "I ordered a laptop 3 days ago but haven't received shipping confirmation"})
print(response.content) # Use .content to access the response text

Hi there! I’m sorry to hear you haven’t received a shipping confirmation yet. Let’s get that sorted out right away.

Could you please provide your order number or the email address you used to place the order? With that information, I can:

1. **Check the status** of your shipment in our system.
2. **Verify** that the confirmation email was sent and wasn’t caught in spam/junk folders.
3. **Send a fresh confirmation** if necessary.

If you’d like, I can also walk you through how to track your order on our website once the package is on its way.

Thank you for choosing TechStore, and I’ll make sure we get your laptop’s shipping details to you as soon as possible!


### Advanced Customer Support Features

In [ ]:
# Create a knowledge base for common issues
knowledge_base = {
    "shipping": "Standard shipping takes 3-5 business days. Express shipping takes 1-2 business days.",
    "returns": "You can return items within 30 days of purchase. Items must be in original condition.",
    "warranty": "All electronics come with a 1-year manufacturer warranty."
}

In [ ]:
# Enhanced support chain with knowledge lookup
def enhanced_support_response(question):
    # Simple keyword matching for demo
    for key, info in knowledge_base.items():
        if key.lower() in question.lower():
            return f"Based on our policy: {info}\n\nLet me provide more specific help with your question."
    return "Let me help you with that question."

In [ ]:
# Test enhanced support
kb_info = enhanced_support_response("What's your return policy?")
full_response = support_chain.invoke({"input": f"{kb_info}\n\nCustomer question: What's your return policy?"}) # Use .invoke()
print(full_response.content) # Use .content to access the response text

Absolutely! Here’s a quick of TechStore’s return policy:

| **Item** | **Timeframe** | **Condition** | **Refund/Replacement** |
|----------|---------------|---------------|------------------------|
| All items | 30 days from purchase or 30 days from delivery | New condition, with original packaging and tags | Full refund (to original payment method) |
| Digital products | 30 days from purchase | Unused | Full refund |
| Opened or used items | 30 days from delivery | In original condition | Full refund *only if* the item is defective or not as described. |

### How to Return an Item
1. **Log in** to your account and go to **Orders**.  
2. Select the item you’d like to return and click **Return**.  
3. Print the prepaid return label that’s emailed to you.  
4. Pack the item securely in its original packaging.  
5. Drop it off at the nearest shipping location.

### What Happens After We Receive the Return
- We’ll inspect the item.
- If it meets the return criteria, we’ll issue a **full re

## Scenario 2: Document Analysis & Summarization

### Real-World Application: Legal Document Review

Let's create a system that can analyze and summarize legal contracts or business documents.

In [ ]:
# Create sample document content (you can upload files in Colab)
sample_contract = """
EMPLOYMENT AGREEMENT
This Employment Agreement is entered into between TechCorp Inc. and John Smith.
Position: Senior Software Engineer
Start Date: January 15, 2024
Salary: $120,000 annually
Benefits: Health insurance, dental coverage, 401k matching up to 4%
Vacation: 20 days annually
Termination: Either party may terminate with 30 days notice
Non-compete: 6 months restriction in same industry within 50 miles
"""

In [ ]:
# Create document analyzer
doc_analyzer_prompt = ChatPromptTemplate.from_template("""
Analyze the following document and provide:
1. Key terms and conditions
2. Important dates and numbers
3. Potential concerns or red flags
4. Brief summary

Document: {document}
""")

In [ ]:
doc_chain = LLMChain(llm=llm, prompt=doc_analyzer_prompt)
analysis = doc_chain.run(document=sample_contract)
print(analysis)

/tmp/ipython-input-4278946011.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  analysis = doc_chain.run(document=sample_contract)


**1. Key Terms & Conditions**

| Element | Detail |
|---------|--------|
| **Parties** | TechCorp Inc. (employer) & John Smith (employee) |
| **Position** | Senior Software Engineer |
| **Start Date** | January 15, 2024 |
| **Compensation** | $120,000 per year (annual salary) |
| **Benefits** | • Health insurance<br>• Dental coverage<br>• 401(k) matching up to 4% of employee contributions |
| **Vacation** | 20 paid days per calendar year |
| **Termination** | Either party may terminate with **30‑day written notice** |
| **Non‑Compete** | • 6‑month restriction on working in the same industry<br>• Geographic scope: within 50 miles of the employer’s principal place of business |

**2. Important Dates & Numbers**

| Item | Value |
|------|-------|
| Start Date | January 15, 2024 |
| Salary | $120,000/yr |
| 401(k) Match | Up to 4% of employee contributions |
| Vacation Days | 20 days/year |
| Termination Notice | 30 days |
| Non‑Compete Duration | 6 months |
| Non‑Compete Radius | 50 miles

### Document Splitting and Processing

In [ ]:
# For larger documents, split them into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

In [ ]:
# Create a document summarization chain
summary_prompt = ChatPromptTemplate.from_template("""
Summarize the following text section concisely:
{text}

Summary:
""")

In [ ]:
summary_chain = LLMChain(llm=llm, prompt=summary_prompt)
chunks = text_splitter.split_text(sample_contract)
summaries = [summary_chain.run(text=chunk) for chunk in chunks]
print("Document summaries:", summaries)

Document summaries: ['TechCorp Inc. hires John\u202fSmith as Senior Software Engineer effective\u202f15\u202fJan\u202f2024, paying\u202f$120\u202fk/year with health, dental, and 401(k) matching up to\u202f4\u202f%. He receives\u202f20\u202fvacation days. Either party may end the contract with\u202f30\u202fdays’ notice. He is restricted from competing in the same industry within\u202f50\u202fmiles for\u202f6\u202fmonths.']


## Scenario 3: Code Review Assistant

### Real-World Application: Automated Code Quality Assessment

Let's build a system that reviews code for best practices, bugs, and improvements.

In [ ]:
# Sample code to review
sample_code = '''
def calculate_discount(price, discount_percent):
    discount = price * discount_percent / 100
    final_price = price - discount
    return final_price

# Usage
price = 100
result = calculate_discount(price, 20)
print(f"Final price: ${result}")
'''

In [ ]:
# Create code review prompt
code_review_prompt = ChatPromptTemplate.from_template("""
Review the following code and provide:
1. Code quality assessment (1-10)
2. Potential bugs or issues
3. Improvement suggestions
4. Best practices recommendations

Code:
{code}
""")

In [ ]:
code_reviewer = LLMChain(llm=llm, prompt=code_review_prompt)
review = code_reviewer.run(code=sample_code)
print(review)

**1. Code Quality Assessment (1–10)**  
**Score: 6/10**

The function is short, straightforward, and works for the simple scenario shown.  
However, it lacks many of the niceties that modern Python code usually contains: type hints, docstrings, input validation, and proper handling of monetary values.

---

**2. Potential Bugs / Issues**

| Issue | Why it matters | Example scenario |
|-------|----------------|------------------|
| **No input validation** | Negative prices or discounts greater than 100 % would produce nonsensical results. | `calculate_discount(-50, 10)` → `-55` (should probably raise an error or clamp). |
| **Floating‑point arithmetic for money** | Binary floating‑point can produce round‑off errors (`0.1 + 0.2 != 0.3`). | `calculate_discount(0.3, 25)` → `0.22499999999999997` instead of `0.225`. |
| **No documentation** | Future readers (or yourself months later) won’t know the intended contract of the function. | Unclear if `discount_percent` is a whole number (e.g., 20

### Security Code Review

In [ ]:
# Security-focused code review
security_prompt = ChatPromptTemplate.from_template("""
Perform a security analysis of this code:
1. Identify security vulnerabilities
2. Check for input validation issues
3. Look for potential injection attacks
4. Suggest security improvements

Code: {code}
""")

In [ ]:
security_reviewer = LLMChain(llm=llm, prompt=security_prompt)
security_analysis = security_reviewer.run(code=sample_code)
print(security_analysis)

## Security Analysis of the Provided Code

| # | Issue | Why it matters | Example of misuse | Suggested fix |
|---|-------|----------------|-------------------|---------------|
| 1 | **No input validation** | The function accepts *any* value for `price` and `discount_percent`.  If an attacker supplies malicious or malformed data, the function can produce nonsensical results or even crash. | `calculate_discount("cheap", -200)` → `TypeError`, `ZeroDivisionError` (if percent > 100 and you later divide by it elsewhere). | *Validate types and ranges.*<br>`if not isinstance(price, (int, float)) or price < 0: raise ValueError(...)`<br>`if not isinstance(discount_percent, (int, float)) or not (0 <= discount_percent <= 100): raise ValueError(...)` |
| 2 | **No type checking** | The function is used in an environment where `price` or `discount_percent` might be strings (e.g., from a web form). This leads to *type errors* or *incorrect calculations*. | `calculate_discount("50", "10")` → `TypeErro

## Scenario 4: Content Generation Pipeline

### Real-World Application: Marketing Content Creation

Let's create a pipeline that generates marketing content for different platforms.

In [ ]:
# Content generation for different platforms
platforms = {
    "twitter": "Create a Twitter post (280 chars max)",
    "linkedin": "Create a LinkedIn professional post",
    "blog": "Create a blog post introduction"
}

In [ ]:
# Base content prompt
content_prompt = ChatPromptTemplate.from_template("""
Create {platform_type} content about: {topic}
Target audience: {audience}
Tone: {tone}
Include relevant hashtags where appropriate.
""")

In [ ]:
content_generator = LLMChain(llm=llm, prompt=content_prompt)

In [ ]:
# Generate content for multiple platforms
topic = "AI in healthcare"
audience = "healthcare professionals"
tone = "professional yet engaging"

In [ ]:
# Generate for different platforms
for platform, description in platforms.items():
    content = content_generator.run(
        platform_type=description,
        topic=topic,
        audience=audience,
        tone=tone
    )
    print(f"\n{platform.upper()} CONTENT:\n{content}\n{'-'*50}")


TWITTER CONTENT:
AI is reshaping diagnostics, personalized medicine, and population health. Real‑time imaging analysis and predictive analytics empower clinicians to make faster, evidence‑based decisions. Embrace AI responsibly to enhance patient outcomes. #AIinHealth #MedTech
--------------------------------------------------

LINKEDIN CONTENT:
🚀 **AI in Healthcare: Transforming Patient Care & Clinical Workflows**  
*(For colleagues in medicine, nursing, administration, and allied health)*  

In the past year, AI has moved from research labs to bedside, offering tangible benefits across the continuum of care. Here are the key take‑aways that matter to us as frontline professionals:

1. **Early & Accurate Diagnostics**  
   - Deep‑learning algorithms now detect diabetic retinopathy, lung nodules, and skin cancers with sensitivities that rival, and sometimes exceed, expert radiologists.  
   - Real‑time image analysis during procedures (e.g., endoscopy) flags subtle abnormalities the h

### Content Optimization Chain

In [ ]:
# Create content optimization chain
optimization_prompt = ChatPromptTemplate.from_template("""
Optimize this content for better engagement:
1. Make it more compelling
2. Add emotional hooks
3. Include call-to-action
4. Improve readability

Original content: {content}
""")

In [ ]:
optimizer = LLMChain(llm=llm, prompt=optimization_prompt)
sample_content = "Our new product uses AI to help doctors."
optimized = optimizer.run(content=sample_content)
print(optimized)

**Transform the way you care for patients**  

Imagine a tool that instantly sifts through mountains of data, flags the subtle red‑flags you might miss, and gives you the confidence to make the right decision—faster. Our new AI‑powered platform does just that, freeing up your time so you can focus on what truly matters: your patients.  

- **Save hours each week** – let the AI handle the heavy lifting of data analysis.  
- **Catch early warning signs** – get real‑time alerts that help prevent complications.  
- **Elevate patient outcomes** – deliver evidence‑based care with pinpoint accuracy.  

Feel the relief of less paperwork, more patient smiles, and a clearer path to better health.  

**Ready to experience the difference?**  
👉 **Book a free, no‑commitment demo today** and see how our AI can become your new best ally.  

*(Slots are limited—reserve yours now!* )


## Scenario 5: Data Analysis & Insights

### Real-World Application: Sales Data Analysis

Let's create a system that analyzes sales data and provides business insights.

In [ ]:
# Sample sales data
import pandas as pd
sales_data = {
    'month': ['Jan', 'Feb', 'Mar', 'Apr', 'May'],
    'revenue': [50000, 65000, 45000, 70000, 80000],
    'customers': [200, 250, 180, 280, 320]
}
df = pd.DataFrame(sales_data)

In [ ]:
# Convert data to text for analysis
data_summary = df.to_string()
print("Sales Data:")
print(data_summary)

Sales Data:
  month  revenue  customers
0   Jan    50000        200
1   Feb    65000        250
2   Mar    45000        180
3   Apr    70000        280
4   May    80000        320


In [ ]:
# Create data analysis prompt
analysis_prompt = ChatPromptTemplate.from_template("""
Analyze this sales data and provide:
1. Key trends and patterns
2. Month-over-month growth analysis
3. Customer acquisition insights
4. Recommendations for improvement
5. Predicted next month performance

Data:
{data}
""")

In [ ]:
data_analyzer = LLMChain(llm=llm, prompt=analysis_prompt)
insights = data_analyzer.run(data=data_summary)
print(insights)

**1. Key Trends & Patterns**

| Month | Revenue | Customers | Revenue Growth % | Customer Growth % |
|-------|--------|-----------|------------------|-------------------|
| Jan   | $50,000 | 200 | – | – |
| Feb   | $65,000 | 250 | **+30 %** | **+25 %** |
| Mar   | $45,000 | 180 | **‑30.8 %** | **‑28 %** |
| Apr   | $70,000 | 280 | **+55.6 %** | **+55.6 %** |
| May   | $80,000 | 320 | **+14.3 %** | **+14.3 %** |

**Observations**

1. **Revenue & Customer Movements Co‑Shaped** – Every month’s revenue swing closely mirrors the customer swing, suggesting that most revenue variability comes from volume changes rather than price/average order value (AOV) changes.
2. **Quarter‑over‑Quarter Volatility** – A sharp dip in March (‑30 %) followed by a robust rebound in April (≈+55 %) points to a cyclical pattern—perhaps a seasonal dip (e.g., end‑of‑Q1 slowdown) and a post‑holiday surge.
3. **Steady Growth After the Dip** – From April to May, growth slows to a more modest +14 % for both metrics, in

### Automated Report Generation

In [ ]:
# Create executive summary generator
report_prompt = ChatPromptTemplate.from_template("""
Create an executive summary report based on this analysis:
{analysis}

Format as:
- Executive Summary (2-3 sentences)
- Key Metrics
- Action Items
- Risk Assessment
""")

In [ ]:
report_generator = LLMChain(llm=llm, prompt=report_prompt)
executive_report = report_generator.run(analysis=insights)
print("EXECUTIVE REPORT:")
print(executive_report)

EXECUTIVE REPORT:
**Executive Summary**  
Revenue and customer volume have been tightly coupled over the past five months, with a sharp March dip followed by a robust April rebound and a modest May plateau. If the outlined retention, upsell, and referral tactics are executed, June revenue could rise to **$85–$90 k** and the customer base to **340–350**, enabling a 10–15 % lift over the next six months.

---

### Key Metrics  

| Metric | Jan | Feb | Mar | Apr | May | Trend |
|--------|-----|-----|-----|-----|-----|-------|
| Revenue | $50 k | $65 k | $45 k | $70 k | $80 k | +30 % → –30.8 % → +55.6 % → +14.3 % |
| Customers | 200 | 250 | 180 | 280 | 320 | +25 % → –28 % → +55.6 % → +14.3 % |
| Avg. Order Value | $250 | $260 | $250 | $250 | $250 | Stable |
| Revenue per Customer (average) | $250 | $260 | $250 | $250 | $250 | Stable |
| Forecast (June) | – | – | – | – | – | Revenue $80.8 k, Customers 327 |

*The revenue‑to‑customer ratio remains constant, indicating volume is the primary d

## Advanced Features

### Chain of Thought Reasoning

In [ ]:
# Complex problem-solving with chain of thought
cot_prompt = ChatPromptTemplate.from_template("""
Solve this business problem step by step:
Problem: {problem}

Think through this systematically:
1. First, identify the key issues
2. Then, consider possible solutions
3. Finally, recommend the best approach

Let's work through this step by step:
""")

In [ ]:
cot_chain = LLMChain(llm=llm, prompt=cot_prompt)
problem = "Our customer acquisition cost has increased 40% while retention dropped 15%"
solution = cot_chain.run(problem=problem)
print(solution)

## 1. Identify the Key Issues  

| Category | What’s Happening | Why It Matters | Immediate Questions |
|----------|------------------|----------------|---------------------|
| **Customer Acquisition Cost (CAC)** | ↑ 40% in the last period | Higher CAC erodes profit margins; the business is spending more to grow the same (or smaller) customer base. | • Which channels drove the spike?<br>• Are bids, targeting or creative quality deteriorating?<br>• Is the sales cycle longer now? |
| **Retention / Churn** | Drop of 15% in retained customers | Losing customers costs far more than acquiring new ones; a 15% drop can wipe out the new growth you’re trying to achieve. | • At what stage in the lifecycle are customers leaving?<br>• Are there product, service or pricing pain points?<br>• How does churn compare with industry benchmarks? |
| **Customer Lifetime Value (LTV)** | Likely falling (or stagnating) | If LTV is not keeping pace with CAC, the business model is unsustainable. | • What is the 

### Multi-Agent Workflow

In [ ]:
# Create different "expert" agents
experts = {
    "marketing": "You are a marketing expert focused on customer acquisition and brand strategy.",
    "finance": "You are a finance expert focused on cost optimization and revenue growth.",
    "operations": "You are an operations expert focused on efficiency and process improvement."
}

In [ ]:
# Get insights from different experts
def get_expert_opinion(expert_type, question):
    expert_prompt = ChatPromptTemplate.from_template(f"""
    {experts[expert_type]}

    Question: {{question}}

    Provide your expert opinion and specific recommendations:
    """)
    expert_chain = LLMChain(llm=llm, prompt=expert_prompt)
    return expert_chain.run(question=question)

In [ ]:
# Consult multiple experts
business_question = "How should we respond to a 20% increase in customer churn?"
for expert in experts.keys():
    opinion = get_expert_opinion(expert, business_question)
    print(f"\n{expert.upper()} EXPERT OPINION:")
    print(opinion)
    print("-" * 60)


MARKETING EXPERT OPINION:
**Executive Summary**  
A 20 % surge in churn is a red‑flag that indicates a shift in customer perception, market dynamics, or product/service fit. The response must be **data‑driven, customer‑centric, and tightly integrated with your brand strategy**.  
Below is a phased playbook—*Immediate, Short‑Term, and Long‑Term*—that blends tactical retention work with strategic brand positioning to reverse churn, protect revenue, and ultimately fuel sustainable growth.

---

## 1. Diagnose the Root Causes (Day 1–7)

| Action | Why It Matters | How to Do It |
|--------|----------------|--------------|
| **Collect & analyze churn data** | Pinpoints which segments, channels, or touchpoints are most affected. | Pull churn logs; segment by tenure, cohort, product line, acquisition channel, geography, and support ticket volume. |
| **Run exit‑survey & NPS follow‑ups** | Uncovers specific pain points (price, feature gaps, service quality). | Automate a 2‑question exit survey

### Error Handling and Validation


In [ ]:
def safe_llm_call(chain, **kwargs):
    try:
        result = chain.run(**kwargs)
        return {"success": True, "result": result}
    except Exception as e:
        return {"success": False, "error": str(e)}

In [ ]:
# Example with error handling
safe_result = safe_llm_call(content_generator,
                          platform_type="Twitter post",
                          topic="Machine Learning",
                          audience="developers",
                          tone="technical")
print(safe_result)

{'success': True, 'result': 'Optimizing your neural nets? Remember batch normalization reduces internal covariate shift, speeding up convergence. Pair it with Adam and a cosine annealing scheduler for smoother training curves. #MachineLearning #DeepLearning #Python #AI #DataScience #ML'}


## Best Practices & Tips

### 1. Prompt Engineering Tips
- Be specific and clear in your instructions
- Use examples when possible
- Break complex tasks into smaller steps
- Include context and constraints

### 2. Performance Optimization
- Use appropriate chunk sizes for document processing
- Implement caching for repeated queries
- Monitor token usage to control costs
- Use streaming for real-time applications

### 3. Error Handling
- Always implement try-catch blocks
- Validate inputs before processing
- Have fallback responses ready
- Log errors for debugging

### 4. Security Considerations
- Never expose API keys in code
- Validate and sanitize user inputs
- Implement rate limiting
- Use environment variables for sensitive data